In [1]:
import ccxt
import datetime
import pandas as pd

# Initialize the OKX exchange
exchange = ccxt.okx({
    'rateLimit': 1000,  # Rate limit (can be adjusted)
    'enableRateLimit': True,
})

# Set the symbols and date range
symbols = ['BTC/USDT', 'ETH/USDT']  # Note: USDT/USDT may not be a valid pair; usually just USDT against other pairs
start_date = '2024-10-04T00:00:00Z'
end_date = '2024-10-04T23:59:59Z'

# Function to fetch historical data
def fetch_data(symbol, start_date, end_date):
    since = exchange.parse8601(start_date)
    data = []
    while since < exchange.parse8601(end_date):
        try:
            ohlcvs = exchange.fetch_ohlcv(symbol, timeframe='1s', since=since, limit=1000)
            if not ohlcvs:
                break
            data.extend(ohlcvs)
            # Update 'since' to fetch the next batch
            since = ohlcvs[-1][0] + 1000  # Offset by 1s in milliseconds
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            break
    return data

# Fetch and store data for each symbol
dataframes = {}
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    raw_data = fetch_data(symbol, start_date, end_date)
    if raw_data:
        df = pd.DataFrame(raw_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        dataframes[symbol] = df

# Save to CSV files
for symbol, df in dataframes.items():
    filename = f"{symbol.replace('/', '_')}_2024-10-04.csv"
    df.to_csv(filename, index=False)
    print(f"Data for {symbol} saved to {filename}")

print("Data fetching complete.")

Fetching data for BTC/USDT...
Fetching data for ETH/USDT...
Data for BTC/USDT saved to BTC_USDT_2024-10-04.csv
Data for ETH/USDT saved to ETH_USDT_2024-10-04.csv
Data fetching complete.
